In [ ]:
#Importing the libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
import numpy as np
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

In [ ]:
# Parameters
batch_size = 32
img_size = (320, 320)

# Function to create a dataset from directories with data augmentation
def create_dataset(directory, is_train=False):
    if is_train:
        # Data augmentation for the training dataset
        data_augmentation = tf.keras.Sequential([
            layers.RandomFlip('horizontal'),
            layers.RandomRotation(0.2),
            layers.RandomZoom(0.2),
            layers.RandomContrast(0.2),
        ])
        dataset = tf.keras.utils.image_dataset_from_directory(
            directory,
            image_size=img_size,
            batch_size=batch_size
        )
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
    else:
        dataset = tf.keras.utils.image_dataset_from_directory(
            directory,
            image_size=img_size,
            batch_size=batch_size
        )
    return dataset

# Load datasets with augmentation for training
train_dataset = create_dataset("/content/drive/MyDrive/dataset/train", is_train=True)
valid_dataset = create_dataset("/content/drive/MyDrive/dataset/valid", is_train=False)
test_dataset = create_dataset("/content/drive/MyDrive/dataset/test", is_train=False)

# Manually get the class names from the directory (updated part)
dataset_dir = "/content/drive/MyDrive/dataset/train"  # Path to your training dataset
class_names = sorted(os.listdir(dataset_dir))  # Sort the subdirectories alphabetically

# Print the class names to verify
print("Class Names:", class_names)

# Prefetch datasets for performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Calculate class weights to handle imbalance
train_labels = np.concatenate([y.numpy() for _, y in train_dataset], axis=0)
class_weights = class_weight.compute_class_weight(
    'balanced', classes=np.unique(train_labels), y=train_labels
)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class Weights:", class_weights_dict)

# Build your own custom model (no pre-trained weights)
model = models.Sequential([
    # Conv Layer 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(320, 320, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),

    # Conv Layer 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),

    # Conv Layer 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),

    # Fully connected layers
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')  # Output layer with the number of classes
])

# Compile the model with learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4, decay_steps=1000, decay_rate=0.9
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=30,
    class_weight=class_weights_dict,
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc}")

# Generate predictions and calculate metrics
y_true = np.concatenate([y.numpy() for _, y in test_dataset], axis=0)
y_pred = np.argmax(model.predict(test_dataset), axis=1)

# Classification Report
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Save results
results = {
    "accuracy": test_acc,
    "classification_report": report,
    "confusion_matrix": conf_matrix.tolist()
}

with open("results.json", "w") as f:
    json.dump(results, f, indent=4)

# Save the trained model
model.save("oral_disease_custom_model.keras")

print("Results and model saved successfully.")

# Visualization: Plot F1-Score, Precision, Recall for each class
f1_scores = [report[class_name]["f1-score"] for class_name in class_names]
precision = [report[class_name]["precision"] for class_name in class_names]
recall = [report[class_name]["recall"] for class_name in class_names]

plt.figure(figsize=(10, 5))
x = np.arange(len(class_names))
width = 0.2

fig, ax = plt.subplots()
rects1 = ax.bar(x - width, f1_scores, width, label='F1 Score')
rects2 = ax.bar(x, precision, width, label='Precision')
rects3 = ax.bar(x + width, recall, width, label='Recall')

ax.set_ylabel('Scores')
ax.set_title('Classification Report Metrics')
ax.set_xticks(x)
ax.set_xticklabels(class_names, rotation=45, ha="right")
ax.legend()

plt.tight_layout()
plt.savefig('metrics_plot.png')  # Save plot as image
plt.show()

# Confusion Matrix Visualization as Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.savefig('confusion_matrix.png')  # Save confusion matrix plot
plt.show()

# Create a Zip file to download the model and results
with zipfile.ZipFile('model_and_results.zip', 'w') as zipf:
    zipf.write('oral_disease_custom_model.keras')
    zipf.write('results.json')
    zipf.write('metrics_plot.png')
    zipf.write('confusion_matrix.png')

print("All files saved and zipped successfully.")


In [ ]:
from google.colab import files
files.download('model_and_results.zip')